# 월, 일별 일출, 일몰 시간뽑기

In [1]:
# preprocessing
import numpy as np
import pandas as pd
import tqdm
import os
import glob

# 경고 무시
import warnings
warnings.filterwarnings('ignore')

In [2]:
# 년월일 데이터 분리
def splitmmdd(df, datecolumn):
    df[datecolumn] = df[datecolumn].astype(str).str.zfill(6)
    df['mm'] = df[datecolumn].str.slice(4, 6).astype(int)
    df['dd'] = df[datecolumn].str.slice(6, 8).astype(int)
    return df.drop(datecolumn, axis = 1)

In [3]:
def make_merge_xlsx(path, names):
    # 파일 불러오기
    files = glob.glob(path + "*.xlsx")

    # 빈 데이터프레임 만들기
    excel = pd.DataFrame()
    
    # 데이터 하나씩 불러와서 붙이기
    for file_name in files:
        df = pd.read_excel(file_name, names = names)
        excel = excel.append(df, ignore_index=False)
        
    return excel

In [4]:
# 경로 설정
path = '../data/sun/'

# 이름 설정
names = ['date', '음력', 'sunrise', '남중', 'sunset'] + ['nouse'] * 9

In [5]:
# 통합하기
df = make_merge_xlsx(path, names)

In [6]:
# 필요한 데이터만 추출하기
makesun = df[df['date'].notnull()].iloc[:,[0, 2, 4]]

In [7]:
# 2월 29일 2020년으로 직접 조사해서 대입
makesun.loc[len(makesun)] = [20200229, '07:04', '18:24']

In [8]:
# 년도 삭제 및 월, 일 분리
sun = splitmmdd(makesun, 'date')

In [9]:
# 일출, 일말 월/일 나누기
sun[['sunriseh', 'sunrisem']] = sun['sunrise'].str.split(':', expand=True)
sun[['sunseth', 'sunsetm']] = sun['sunset'].str.split(':', expand=True)

In [10]:
# 데이터 속성 int로 바꾸기
finalsun = sun[['mm', 'dd', 'sunriseh', 'sunrisem', 'sunseth', 'sunsetm']].astype(int)

In [11]:
# 데이터 저장하기
finalsun.to_csv('../data/sun.csv', index=False)